In [64]:
%pip install pandas numpy scikit-learn boto3 langchain langchain_aws tqdm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

df = pd.read_csv('data/marks.csv')

In [15]:
df['Time'] = pd.to_datetime(df['Time'], format='%Y%m%d%H%M%S')
df['User'] = df['User'].str.replace('@stu.vtc.edu.hk', '')
df['Test'] = df['Test'].str.replace('ProjectTestsLib.', '')
df['User'] = df['User'].astype(str)
df = df[df['User'].str.match(r'^\d{9}$')]

In [16]:
df.to_csv('output/df.csv', index=False)
df.head()

,Test,User,Marks,Time
0,T01_CredentialTest.Test01_ValidCredential,230142256,10,2024-05-28 06:47:33
1,T02_VpcTest.Test01_VpcExist,230142256,10,2024-05-28 06:52:43
2,T02_VpcTest.Test02_VpcOf4Subnets,230142256,10,2024-05-28 06:57:54
3,T02_VpcTest.Test03_VpcOf5RouteTable,230142256,10,2024-05-28 09:12:30
4,T02_VpcTest.Test04_VpcMainRouteTable,230142256,10,2024-05-29 11:15:46


In [17]:
pivot_table = df.pivot_table(index='User', columns='Test', values='Time')
pivot_table = pivot_table.rename(columns=lambda x: x.replace('ProjectTestsLib.', ''))

In [18]:
pivot_table = pivot_table.sort_values(by='User')

In [19]:
pivot_table.tail()

Test,T01_CredentialTest.Test01_ValidCredential,T02_VpcTest.Test01_VpcExist,T02_VpcTest.Test02_VpcOf4Subnets,T02_VpcTest.Test03_VpcOf5RouteTable,T02_VpcTest.Test04_VpcMainRouteTable,T02_VpcTest.Test05_VpcPublicRouteTables,T02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints,T02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints,T02_VpcTest.Test08_VpcIsolatedRouteTables,T02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables,...,T09_ElasticLoadBalancingTest.Test01_ElasticLoadBalancerExist,T09_ElasticLoadBalancingTest.Test02_ElasticLoadBalancerInternetFacing,T09_ElasticLoadBalancingTest.Test03_ElasticLoadBalancerInTwoPublicSubnet,T09_ElasticLoadBalancingTest.Test04_ElasticLoadBalancerListener,T09_ElasticLoadBalancingTest.Test05_ElasticLoadBalancerListenerDefaultAction,T09_ElasticLoadBalancingTest.Test06_ElasticLoadBalancerWithTwoTargetGroups,T09_ElasticLoadBalancingTest.Test07_ElasticLoadBalancerWithLambdaTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test08_ElasticLoadBalancerWithIpTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test09_ElasticLoadBalancerListenerDefaultRule,T09_ElasticLoadBalancingTest.Test10_ElasticLoadBalancerDummyRule
User,,,,,,,,,,,,,,,,,,,,,
230351785,2024-05-20 07:24:29,2024-05-21 04:07:55,2024-05-21 04:23:37,2024-05-21 05:34:47,2024-05-21 05:35:52,2024-05-21 05:43:36,2024-05-21 06:10:28,2024-05-21 06:23:18,2024-05-21 07:19:59,2024-05-21 07:21:42,...,2024-06-06 14:47:54,2024-06-06 14:50:38,2024-06-06 14:51:51,2024-06-06 14:52:31,2024-06-06 14:53:25,2024-06-11 05:32:35,2024-06-11 05:45:59,2024-06-11 05:47:14,2024-06-11 06:23:49,2024-06-11 07:57:44
230353858,2024-05-27 15:22:56,2024-05-27 15:25:48,2024-05-27 15:28:26,2024-05-27 15:37:55,2024-05-27 15:53:18,2024-05-27 15:56:52,2024-05-27 16:03:45,2024-05-27 16:07:58,2024-05-27 16:10:49,2024-05-27 16:11:14,...,2024-05-28 15:30:58,2024-05-28 15:31:40,2024-05-28 15:32:27,2024-05-28 15:33:06,2024-05-28 15:33:39,2024-05-28 16:00:04,2024-05-28 16:02:12,2024-05-28 16:03:24,2024-05-28 16:13:23,2024-05-28 16:16:25
230389216,2024-05-28 13:33:48,2024-05-28 14:24:45,2024-05-28 14:31:00,2024-05-28 15:27:27,2024-05-29 02:20:33,2024-05-29 02:15:30,2024-05-29 02:26:52,2024-05-29 05:28:46,2024-05-29 10:43:35,2024-05-29 10:46:03,...,2024-06-02 10:08:19,2024-06-02 10:08:42,2024-06-02 13:49:25,2024-06-02 13:55:06,2024-06-02 14:06:57,2024-06-02 14:11:59,2024-06-02 14:12:07,2024-06-02 14:12:13,2024-06-02 16:27:22,2024-06-02 16:46:54
230490969,2024-05-24 08:25:42,2024-05-24 08:39:43,2024-05-24 08:45:23,2024-05-24 08:53:21,2024-05-27 07:42:11,2024-05-27 08:33:32,2024-05-27 09:00:02,2024-05-27 09:03:13,2024-05-28 07:31:54,2024-05-28 07:34:10,...,2024-06-03 07:02:18,2024-06-03 07:03:30,2024-06-03 07:04:36,2024-06-03 07:04:59,2024-06-03 07:06:30,2024-06-03 07:47:47,2024-06-03 08:01:17,2024-06-03 08:02:50,2024-06-03 08:33:57,2024-06-03 08:34:57
230494004,2024-05-27 08:39:18,2024-05-27 08:41:57,2024-05-27 08:50:17,2024-05-27 09:00:36,2024-05-29 01:34:13,2024-05-29 06:09:06,2024-05-29 06:28:32,2024-05-29 06:39:50,2024-05-29 06:51:28,2024-05-29 06:52:56,...,2024-06-05 03:57:58,2024-06-05 03:58:34,2024-06-05 03:58:51,2024-06-05 03:58:56,2024-06-05 03:59:00,2024-06-05 04:05:10,2024-06-05 04:14:04,2024-06-05 04:15:06,2024-06-05 04:18:13,2024-06-05 04:18:22


In [80]:
# Step 1: Create a boolean mask for valid time values
valid_time_mask = pivot_table.apply(lambda x: pd.to_datetime(x, errors='coerce').notnull())

# Step 2: Count the number of valid time values in each row
valid_time_count = valid_time_mask.sum(axis=1)

# Step 3: Create a new dataframe with the "total completed tasks" column
total_tasks_dataframe = pd.DataFrame(index=pivot_table.index)
total_tasks_dataframe['total completed tasks'] = valid_time_count
total_tasks_dataframe.to_csv('output/total_tasks.csv')

In [20]:
pivot_table_rank = pivot_table.rank(method='min', na_option='keep')

In [21]:
pivot_table_rank.head()

Test,T01_CredentialTest.Test01_ValidCredential,T02_VpcTest.Test01_VpcExist,T02_VpcTest.Test02_VpcOf4Subnets,T02_VpcTest.Test03_VpcOf5RouteTable,T02_VpcTest.Test04_VpcMainRouteTable,T02_VpcTest.Test05_VpcPublicRouteTables,T02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints,T02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints,T02_VpcTest.Test08_VpcIsolatedRouteTables,T02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables,...,T09_ElasticLoadBalancingTest.Test01_ElasticLoadBalancerExist,T09_ElasticLoadBalancingTest.Test02_ElasticLoadBalancerInternetFacing,T09_ElasticLoadBalancingTest.Test03_ElasticLoadBalancerInTwoPublicSubnet,T09_ElasticLoadBalancingTest.Test04_ElasticLoadBalancerListener,T09_ElasticLoadBalancingTest.Test05_ElasticLoadBalancerListenerDefaultAction,T09_ElasticLoadBalancingTest.Test06_ElasticLoadBalancerWithTwoTargetGroups,T09_ElasticLoadBalancingTest.Test07_ElasticLoadBalancerWithLambdaTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test08_ElasticLoadBalancerWithIpTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test09_ElasticLoadBalancerListenerDefaultRule,T09_ElasticLoadBalancingTest.Test10_ElasticLoadBalancerDummyRule
User,,,,,,,,,,,,,,,,,,,,,
220131698,80.0,80.0,79.0,78.0,69.0,68.0,77.0,76.0,72.0,71.0,...,73.0,73.0,70.0,71.0,71.0,60.0,61.0,61.0,NaN,58.0
230000997,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0
230021864,54.0,53.0,51.0,47.0,58.0,56.0,55.0,53.0,51.0,49.0,...,43.0,42.0,41.0,41.0,41.0,36.0,37.0,37.0,37.0,37.0
230029966,68.0,67.0,66.0,64.0,55.0,70.0,68.0,67.0,NaN,72.0,...,51.0,51.0,49.0,49.0,49.0,NaN,NaN,NaN,NaN,NaN
230030824,57.0,56.0,56.0,53.0,49.0,50.0,49.0,48.0,45.0,45.0,...,30.0,30.0,29.0,29.0,29.0,32.0,32.0,32.0,35.0,35.0


In [22]:
pivot_table_rank.to_csv('output/pivot_table_rank.csv', index=True)

In [23]:
pivot_table_rank_next = pivot_table_rank.copy()

In [24]:
import numpy as np

# Check if the value is not NaN and add 1
pivot_table_rank_next[pd.notnull(pivot_table_rank_next)] -= 1

In [25]:
pivot_table_rank_next.head()

Test,T01_CredentialTest.Test01_ValidCredential,T02_VpcTest.Test01_VpcExist,T02_VpcTest.Test02_VpcOf4Subnets,T02_VpcTest.Test03_VpcOf5RouteTable,T02_VpcTest.Test04_VpcMainRouteTable,T02_VpcTest.Test05_VpcPublicRouteTables,T02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints,T02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints,T02_VpcTest.Test08_VpcIsolatedRouteTables,T02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables,...,T09_ElasticLoadBalancingTest.Test01_ElasticLoadBalancerExist,T09_ElasticLoadBalancingTest.Test02_ElasticLoadBalancerInternetFacing,T09_ElasticLoadBalancingTest.Test03_ElasticLoadBalancerInTwoPublicSubnet,T09_ElasticLoadBalancingTest.Test04_ElasticLoadBalancerListener,T09_ElasticLoadBalancingTest.Test05_ElasticLoadBalancerListenerDefaultAction,T09_ElasticLoadBalancingTest.Test06_ElasticLoadBalancerWithTwoTargetGroups,T09_ElasticLoadBalancingTest.Test07_ElasticLoadBalancerWithLambdaTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test08_ElasticLoadBalancerWithIpTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test09_ElasticLoadBalancerListenerDefaultRule,T09_ElasticLoadBalancingTest.Test10_ElasticLoadBalancerDummyRule
User,,,,,,,,,,,,,,,,,,,,,
220131698,79.0,79.0,78.0,77.0,68.0,67.0,76.0,75.0,71.0,70.0,...,72.0,72.0,69.0,70.0,70.0,59.0,60.0,60.0,NaN,57.0
230000997,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
230021864,53.0,52.0,50.0,46.0,57.0,55.0,54.0,52.0,50.0,48.0,...,42.0,41.0,40.0,40.0,40.0,35.0,36.0,36.0,36.0,36.0
230029966,67.0,66.0,65.0,63.0,54.0,69.0,67.0,66.0,NaN,71.0,...,50.0,50.0,48.0,48.0,48.0,NaN,NaN,NaN,NaN,NaN
230030824,56.0,55.0,55.0,52.0,48.0,49.0,48.0,47.0,44.0,44.0,...,29.0,29.0,28.0,28.0,28.0,31.0,31.0,31.0,34.0,34.0


In [26]:
import pandas as pd

columns_index={}
for column_name in pivot_table_rank.columns:
    # Initialize a dictionary to store the results
    matched_indices = {}

    # Loop through each row in pivot_table_rank
    for index, row in pivot_table_rank.iterrows():
        value_to_find = row[column_name]
        
        if pd.isnull(value_to_find):
            continue
        # Search for the value in pivot_table_rank_2
        matched_row = pivot_table_rank_next[pivot_table_rank_next[column_name] == value_to_find]
        
        # If the value is found, save the row index (assuming the value is unique)
        if not matched_row.empty:
            matched_index = matched_row.index[0]
            matched_indices[index] = matched_index
    columns_index[column_name] = matched_indices

# matched_indices now contains the row indices from pivot_table_rank_2

In [27]:
columns_index

{'T01_CredentialTest.Test01_ValidCredential': {'220131698': '230242175',
  '230000997': '230279433',
  '230021864': '230262137',
  '230029966': '230235541',
  '230030824': '230109155',
  '230033144': '230389216',
  '230033980': '230034663',
  '230033992': '230033980',
  '230034000': '230090894',
  '230034663': '230353858',
  '230035008': '230065744',
  '230036515': '230104504',
  '230037192': '230087933',
  '230037561': '230196873',
  '230038084': '230075983',
  '230039081': '230494004',
  '230039972': '230180642',
  '230040319': '230037561',
  '230042860': '230066821',
  '230042915': '230160471',
  '230044620': '230000997',
  '230045874': '230047321',
  '230046521': '230042860',
  '230047321': '230087343',
  '230049222': '230060712',
  '230054582': '230106221',
  '230056495': '230309903',
  '230056839': '230166592',
  '230059448': '230046521',
  '230060712': '230037192',
  '230065744': '230108158',
  '230066821': '230045874',
  '230070545': '230137314',
  '230074777': '230172003',
  '

In [28]:
columns_index_time = {}
for column_name in pivot_table_rank.columns:
    columns_index_time[column_name] = {}
    matched_indices = columns_index[column_name]
    
    for k,v in matched_indices.items():        
        try:
            time_diff= pivot_table.loc[v][column_name] - pivot_table.loc[k][column_name]             
        except KeyError :
            time_diff = None           
        columns_index_time[column_name][k]=time_diff;

In [29]:
columns_index_time

{'T01_CredentialTest.Test01_ValidCredential': {'220131698': Timedelta('0 days 14:41:23'),
  '230000997': Timedelta('0 days 03:17:01'),
  '230021864': Timedelta('0 days 00:09:01'),
  '230029966': Timedelta('0 days 00:04:11'),
  '230030824': Timedelta('0 days 01:13:12'),
  '230033144': Timedelta('0 days 01:12:57'),
  '230033980': Timedelta('0 days 03:01:44'),
  '230033992': Timedelta('0 days 00:00:45'),
  '230034000': Timedelta('0 days 02:53:59'),
  '230034663': Timedelta('0 days 03:26:33'),
  '230035008': Timedelta('0 days 00:25:15'),
  '230036515': Timedelta('0 days 02:49:17'),
  '230037192': Timedelta('0 days 00:16:24'),
  '230037561': Timedelta('0 days 12:36:40'),
  '230038084': Timedelta('0 days 00:03:50'),
  '230039081': Timedelta('0 days 00:12:50'),
  '230039972': Timedelta('0 days 00:08:37'),
  '230040319': Timedelta('0 days 00:01:03'),
  '230042860': Timedelta('0 days 03:54:49'),
  '230042915': Timedelta('0 days 10:29:38'),
  '230044620': Timedelta('0 days 17:10:51'),
  '2300458

In [30]:
import pandas as pd

# Transpose the dictionary
# transposed_dict = {column: {index: value for index, value in values.items()} for column, values in columns_index_time.items()}

# Convert the transposed dictionary to a DataFrame
df_columns_index_time = pd.DataFrame.from_dict(columns_index_time, orient='index')
df_columns_index_time = df_columns_index_time.transpose()
df_columns_index_time.sort_index()

,T01_CredentialTest.Test01_ValidCredential,T02_VpcTest.Test01_VpcExist,T02_VpcTest.Test02_VpcOf4Subnets,T02_VpcTest.Test03_VpcOf5RouteTable,T02_VpcTest.Test04_VpcMainRouteTable,T02_VpcTest.Test05_VpcPublicRouteTables,T02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints,T02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints,T02_VpcTest.Test08_VpcIsolatedRouteTables,T02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables,...,T09_ElasticLoadBalancingTest.Test06_ElasticLoadBalancerWithTwoTargetGroups,T09_ElasticLoadBalancingTest.Test07_ElasticLoadBalancerWithLambdaTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test08_ElasticLoadBalancerWithIpTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test10_ElasticLoadBalancerDummyRule,T03_SecurityGroupTest.Test03_WebLambdaSecurityGroupRules,T06_LambdaTest.Test03_FunctionEnvironmentVariables,T06_LambdaTest.Test07_FunctionResourcePolicy,T07_CloudWatchTest.Test03_BatchProcessLogGroupAlarmForError,T08_AutoscalingTest.Test03_LaunchTemplateSettings,T09_ElasticLoadBalancingTest.Test09_ElasticLoadBalancerListenerDefaultRule
220131698,0 days 14:41:23,4 days 20:03:06,4 days 19:55:04,0 days 17:12:11,0 days 17:31:01,0 days 01:51:50,9 days 16:21:34,9 days 18:05:40,2 days 17:49:37,0 days 06:54:16,...,0 days 03:21:56,0 days 03:15:13,0 days 03:16:05,0 days 03:51:12,NaT,NaT,NaT,NaT,NaT,NaT
230000997,0 days 03:17:01,0 days 03:11:57,0 days 03:11:49,0 days 03:40:00,0 days 02:34:25,0 days 02:51:43,0 days 04:00:37,0 days 03:51:32,0 days 04:01:49,0 days 04:06:06,...,0 days 00:25:11,0 days 11:49:45,0 days 04:52:19,0 days 04:38:44,0 days 00:12:32,0 days 03:30:01,0 days 04:20:03,0 days 04:47:27,0 days 11:21:21,0 days 04:41:59
230021864,0 days 00:09:01,0 days 00:11:33,0 days 00:12:44,0 days 00:03:34,0 days 03:10:56,0 days 01:08:17,0 days 01:08:40,0 days 01:23:03,0 days 01:45:48,0 days 01:45:05,...,0 days 00:39:34,0 days 00:46:20,0 days 00:36:35,0 days 01:31:31,0 days 00:02:31,0 days 07:20:27,0 days 23:53:00,0 days 06:38:35,0 days 07:29:18,0 days 01:26:27
230029966,0 days 00:04:11,0 days 00:00:45,0 days 01:16:28,0 days 00:06:04,0 days 01:13:29,0 days 10:41:11,0 days 03:09:40,0 days 03:06:30,NaT,2 days 11:51:37,...,NaT,NaT,NaT,NaT,NaT,NaT,0 days 13:33:04,NaT,NaT,NaT
230030824,0 days 01:13:12,0 days 01:12:58,0 days 22:52:44,0 days 01:39:05,0 days 14:09:15,0 days 05:44:38,0 days 15:32:33,0 days 06:11:25,0 days 05:30:41,1 days 05:41:07,...,0 days 01:00:49,0 days 00:11:48,0 days 01:28:35,0 days 00:08:23,0 days 01:08:42,0 days 04:08:46,0 days 00:01:12,0 days 06:10:24,0 days 01:06:16,0 days 00:06:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230351785,1 days 06:18:22,0 days 10:28:25,0 days 22:36:21,0 days 21:32:22,0 days 21:51:40,0 days 21:48:51,0 days 21:30:35,0 days 21:19:59,0 days 23:40:34,1 days 07:53:10,...,0 days 00:19:26,0 days 00:08:37,0 days 00:08:19,0 days 23:20:03,0 days 03:41:54,0 days 13:12:18,0 days 17:45:35,0 days 12:51:37,0 days 02:35:36,0 days 18:35:14
230353858,0 days 00:56:19,0 days 00:37:11,0 days 00:40:39,0 days 00:41:01,0 days 00:30:49,0 days 01:00:55,0 days 01:09:01,0 days 01:17:51,0 days 01:57:40,0 days 02:05:30,...,0 days 01:10:49,0 days 01:13:29,0 days 01:15:27,1 days 00:04:06,0 days 04:50:25,0 days 19:28:18,0 days 19:47:07,0 days 19:33:37,0 days 00:49:25,0 days 14:30:26
230389216,0 days 00:41:54,0 days 10:36:49,0 days 00:32:48,0 days 00:12:53,0 days 00:43:58,0 days 01:15:28,0 days 01:03:30,0 days 01:11:04,0 days 01:16:26,0 days 01:15:53,...,0 days 00:41:49,0 days 00:45:51,0 days 00:46:49,0 days 15:48:03,0 days 00:31:16,0 days 05:24:06,0 days 16:15:48,0 days 23:01:02,0 days 00:06:34,0 days 16:06:35
230490969,2 days 13:03:50,2 days 12:53:17,2 days 12:52:02,2 days 12:46:43,0 days 08:11:07,0 days 07:23:20,0 days 07:03:43,0 days 07:04:45,0 days 05:14:46,0 days 03:22:52,...,1 days 01:57:10,1 days 01:47:09,1 days 01:47:32,0 days 22:09:53,0 days 00:06:55,0 days 00:49:43,0 days 01:30:16,0 days 01:40:56,0 days 21:26:26,0 days 22:08:59


In [31]:
df_columns_index_time = df_columns_index_time / pd.Timedelta(hours=1)
df_columns_index_time = df_columns_index_time.round().fillna(0).astype(int)

In [32]:
df_columns_index_time.head()

,T01_CredentialTest.Test01_ValidCredential,T02_VpcTest.Test01_VpcExist,T02_VpcTest.Test02_VpcOf4Subnets,T02_VpcTest.Test03_VpcOf5RouteTable,T02_VpcTest.Test04_VpcMainRouteTable,T02_VpcTest.Test05_VpcPublicRouteTables,T02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints,T02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints,T02_VpcTest.Test08_VpcIsolatedRouteTables,T02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables,...,T09_ElasticLoadBalancingTest.Test06_ElasticLoadBalancerWithTwoTargetGroups,T09_ElasticLoadBalancingTest.Test07_ElasticLoadBalancerWithLambdaTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test08_ElasticLoadBalancerWithIpTargetGroupHealthCheckSettings,T09_ElasticLoadBalancingTest.Test10_ElasticLoadBalancerDummyRule,T03_SecurityGroupTest.Test03_WebLambdaSecurityGroupRules,T06_LambdaTest.Test03_FunctionEnvironmentVariables,T06_LambdaTest.Test07_FunctionResourcePolicy,T07_CloudWatchTest.Test03_BatchProcessLogGroupAlarmForError,T08_AutoscalingTest.Test03_LaunchTemplateSettings,T09_ElasticLoadBalancingTest.Test09_ElasticLoadBalancerListenerDefaultRule
220131698,15,116,116,17,18,2,232,234,66,7,...,3,3,3,4,0,0,0,0,0,0
230000997,3,3,3,4,3,3,4,4,4,4,...,0,12,5,5,0,4,4,5,11,5
230021864,0,0,0,0,3,1,1,1,2,2,...,1,1,1,2,0,7,24,7,7,1
230029966,0,0,1,0,1,11,3,3,0,60,...,0,0,0,0,0,0,14,0,0,0
230030824,1,1,23,2,14,6,16,6,6,30,...,1,0,1,0,1,4,0,6,1,0


In [33]:
df_columns_index_time.to_csv('output/time_diff.csv')

In [34]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd

sil_score_max = -1 

scaler = StandardScaler()
pivot_table_cluster = pivot_table.copy()
df_scaled = scaler.fit_transform(pivot_table_cluster)

for n_clusters in range(10,20):
  model = KMeans(n_clusters = n_clusters, init='k-means++', max_iter=100, n_init=1)
  labels = model.fit_predict(df_scaled)
  sil_score = silhouette_score(df_scaled, labels)
  print("The average silhouette score for %i clusters is %0.2f" %(n_clusters,sil_score))
  if sil_score > sil_score_max:
    sil_score_max = sil_score
    best_n_clusters = n_clusters    

print("The best number of clusters is %i" %best_n_clusters)

# Step 3 & 4: Choose and configure the clustering algorithm
kmeans = KMeans(n_clusters=best_n_clusters)  # Example: choosing 3 clusters

# Step 5: Fit the model
kmeans.fit(df_scaled)

pivot_table_cluster = pivot_table.copy()
# Step 6: Assign clusters back to the DataFrame
pivot_table_cluster['cluster'] = kmeans.labels_
pivot_table_cluster = pivot_table_cluster[['cluster'] + [col for col in df_columns_index_time.columns if col != 'cluster']]

# Step 7: Analyze the results
print(pivot_table_cluster.head())  # Print the first few rows to check the cluster column

The average silhouette score for 10 clusters is 0.51
The average silhouette score for 11 clusters is 0.51
The average silhouette score for 12 clusters is 0.66
The average silhouette score for 13 clusters is 0.61
The average silhouette score for 14 clusters is 0.36
The average silhouette score for 15 clusters is 0.49
The average silhouette score for 16 clusters is 0.65
The average silhouette score for 17 clusters is 0.59
The average silhouette score for 18 clusters is 0.46
The average silhouette score for 19 clusters is 0.57
The best number of clusters is 12
Test       cluster T01_CredentialTest.Test01_ValidCredential  \
User                                                           
220131698        4                       2024-06-04 10:12:09   
230000997        0                       2024-05-14 06:31:55   
230021864        4                       2024-05-30 06:35:44   
230029966        7                       2024-06-03 05:47:26   
230030824        4                       2024-05-30 

In [57]:
prompt_contexts = {}
for index, row in pivot_table.iterrows():
    completion_time_row_dict = row.to_dict()
    completion_time = '\n'.join([f"{k}: {v if not pd.isna(v) else 'Unable to finish'}" for k, v in completion_time_row_dict.items()])
    task_ranking_row = pivot_table_rank.loc[index]
    task_ranking_row_dict = task_ranking_row.to_dict()
    task_ranking = '\n'.join([f"{k}: {int(v) if not np.isnan(v) else 'Unable to finish'}" for k, v in task_ranking_row_dict.items()])

    try:
        df_columns_index_time_row = df_columns_index_time.loc[index]
        df_columns_index_time_row_dict = df_columns_index_time_row.to_dict()
        time_diff = '\n'.join([f"{k}: {v if not pd.isna(v) else ''}" for k, v in df_columns_index_time_row_dict.items()])
    except KeyError:
        time_diff = 'Special case, he is the last one to finish the task, so there is no time difference for the next student.'

    prompt_contexts[index] = f"""
The time completed for each AWS task:
{completion_time}
The time ranking for each AWS task in his class:
{task_ranking}
The time difference between the student and the next student completed the task in his class:
{time_diff}
"""
prompt_contexts

{'220131698': '\nThe time completed for each AWS task:\nT01_CredentialTest.Test01_ValidCredential: 2024-06-04 10:12:09\nT02_VpcTest.Test01_VpcExist: 2024-06-05 12:08:38\nT02_VpcTest.Test02_VpcOf4Subnets: 2024-06-05 12:34:01\nT02_VpcTest.Test03_VpcOf5RouteTable: 2024-06-05 12:34:27\nT02_VpcTest.Test04_VpcMainRouteTable: 2024-06-05 12:34:59\nT02_VpcTest.Test05_VpcPublicRouteTables: 2024-06-05 13:15:33\nT02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints: 2024-06-12 22:08:53\nT02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints: 2024-06-12 20:28:10\nT02_VpcTest.Test08_VpcIsolatedRouteTables: 2024-06-12 21:02:42\nT02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables: 2024-06-12 20:31:07\nT03_SecurityGroupTest.Test01_3SecurityGroups: 2024-06-12 21:05:29\nT03_SecurityGroupTest.Test02_AlbSecurityGroupRules: 2024-06-12 20:31:32\nT03_SecurityGroupTest.Test03_WebLambdaSecurityGroupRules: Unable to finish\nT03_SecurityGroupTest.Test04_DatabaseSecurityGroupRules: 2024-06-12 21:36:55\nT04_SqsSnsTest.Test01_3

In [58]:
import numpy as np

# Create a dictionary to store the results
valid_values_dict = {}

# Iterate over each column in the pivot table
for col in pivot_table.columns:
    # Count the number of valid values in the column
    num_valid_values = np.count_nonzero(~np.isnan(pivot_table[col].values))
    
    # Store the result in the dictionary
    valid_values_dict[col] = num_valid_values

# Print the dictionary
completed_no_dict_str = '\n'.join([f"{k}: {v}" for k, v in valid_values_dict.items()])
completed_no_dict_str

'T01_CredentialTest.Test01_ValidCredential: 87\nT02_VpcTest.Test01_VpcExist: 86\nT02_VpcTest.Test02_VpcOf4Subnets: 86\nT02_VpcTest.Test03_VpcOf5RouteTable: 85\nT02_VpcTest.Test04_VpcMainRouteTable: 84\nT02_VpcTest.Test05_VpcPublicRouteTables: 82\nT02_VpcTest.Test06_VpcOf2VpcInterfaceEndpoints: 82\nT02_VpcTest.Test07_VpcOf2VpcGatewayEndpoints: 82\nT02_VpcTest.Test08_VpcIsolatedRouteTables: 78\nT02_VpcTest.Test09_Vpc4SubnetsFor4IndependentRouteTables: 78\nT03_SecurityGroupTest.Test01_3SecurityGroups: 81\nT03_SecurityGroupTest.Test02_AlbSecurityGroupRules: 81\nT03_SecurityGroupTest.Test03_WebLambdaSecurityGroupRules: 70\nT03_SecurityGroupTest.Test04_DatabaseSecurityGroupRules: 73\nT04_SqsSnsTest.Test01_3SqsQueues: 81\nT04_SqsSnsTest.Test02_ToBeProcessQueuesFiFo: 81\nT04_SqsSnsTest.Test03_ToBeProcessQueuesVisibilityTimeout: 80\nT04_SqsSnsTest.Test04_ToBeProcessQueueSetsDeadLetterQueue: 81\nT04_SqsSnsTest.Test05_ErrorTopicSubscribesToErrorQueue: 78\nT05_DatabaseTest.Test01_DatabaseSecrets: 

In [63]:
import datetime
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_aws import BedrockLLM

model = BedrockLLM(
    credentials_profile_name="default",
    model_id="amazon.titan-text-premier-v1:0",
    region_name="us-east-1",
    model_kwargs={"maxTokenCount": 1024, "temperature": 0.5, "topP": 0.95}
)

prompt = PromptTemplate.from_template(
    """
You are a teacher who wants to provide feedback to a student. The student has completed the following AWS tasks:

Scenarios :
    You are going to build an application on AWS based on serverless architecture requirements provided, which contains an Amazon SQS queue that is subscribed to an Amazon SNS topic. You have to make sure the system has High Availability, disaster recovery and being Secure for remote access. 
    Some details / hints as to the order you should do this: 
    Create a new VPC with CIDR (10.0.0.0/16)
    Create 4 Subnets across 2 Availability Zones (AZ), each AZ with 1 public subnet and 1 isolated subnet
    Create correct Route Tables & Security Groups for all AWS services included in project
    Create VPC Endpoints and setup Security Group for the Webserver lambda function i.e. S3, DynamoDB, Secret Manager and SQS.
    Create a SQS queue “To_Be_Processed_Queue” with 300 seconds VisibilityTimeout
    Create a SQS queue “Error_Queue”
    Create a SNS topic “ErrorTopic” with SQS “Error_Queue” subscription
    Create Application Load Balancer (ALB) across 2 AZs  
    pointing to WebServer Lambda functions
    Create a S3 bucket
    Deploy Lambda function “WebLambda” for Web Server 
    Set IAM role permission to access Aurora Serverless Database, S3
    Runtime: python3.8
    Deploy Lambda function “WebInitDatabaseLambda” for Database Initialization.
    Set IAM role permission to access Aurora Serverless Database, S3, DynamoDB
    Configure CloudWatch alarms to trigger SNS ErrorTopic
    Create a DynamoDB Table.

Student Performance:
+++++++++++++++++++++++++++++++++++++++++++++++++++
    {context}
+++++++++++++++++++++++++++++++++++++++++++++++++++

For Example, 
"T01_CredentialTest.Test01_ValidCredential", it means "Credential Test for Valid Credential".
"T02_VpcTest.Test01_VpcExist", it means "VPC Test for VPC Exist".


This assignment start at {start} and deadline is {end}.
Today is {now}
For ranking, the class has {no_of_students} students.
The task and total number of students who completed the task are as follows:
{completed_no_dict_str}

Ranking is an important indicator for the student to know his performance.
Higher the value of Time difference implies the student should complete the task at his own alone and this is good!

Based on the completion time and ranking, provide feedback to the student on his performance.
Feedback should be constructive and encouraging.
Don't just list out the completed or failed tasks.
Analyze the completed tasks and failed tasks to provide suggestions in AWS domain to improve his AWS skills according to the tasks he has completed and failed.
Don't say anything else except the feedback in a paragraph.
Response:"""
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

# response = chain.invoke(
#     {
#         "context": prompt_contexts["220131698"],
#         "start": "8 May 2024",
#         "end": "5 July 2024",
#         "now": datetime.datetime.now().strftime("%d %B %Y"),
#         "no_of_students": len(pivot_table),
#         "completed_no_dict_str": completed_no_dict_str
#     })
# print(response)

In [76]:
from time import sleep
import pandas as pd
from tqdm import tqdm  # Import tqdm

response_dict = {}
for item in tqdm(prompt_contexts.items()):
    response = chain.invoke(   {
        "context": item[1],
        "start": "8 May 2024",
        "end": "5 July 2024",
        "now": datetime.datetime.now().strftime("%d %B %Y"),
        "no_of_students": len(pivot_table),
        "completed_no_dict_str": completed_no_dict_str
     })
    response_dict[item[0]] = response
    sleep(5)



100%|██████████| 87/87 [17:03<00:00, 11.77s/it]


ValueError: If using all scalar values, you must pass an index

In [78]:
import pandas as pd
feedback_df = pd.DataFrame(response_dict.items(), columns=['Student ID', 'Feedback'])
feedback_df.to_csv('output/feedback.csv', index=False)